<a href="https://colab.research.google.com/github/TaahirBhorat/Zindi-Hack-Yassir/blob/master/Catboost-SUBMISSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
import matplotlib.pyplot as plt
!pip install -q catboost

     |████████████████████████████████| 64.8MB 61kB/s 


In [ ]:
train = pd.read_csv('/content/drive/My Drive/Zindi/Train.csv')

In [ ]:
train.head()


,ID,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
0,000FLWA8,3.258,36.777,3.003,36.718,39627,2784,0.200067,-0.052816,-0.050092,0.024168,0,2,338,12
1,000RGOAM,3.087,36.707,3.081,36.727,3918,576,0.034477,0.029167,0.027070,0.009644,0,1,344,12
2,001QSGIH,3.144,36.739,3.088,36.742,7265,526,0.089057,-0.006805,0.032986,-0.005816,0,5,327,11
3,002ACV6R,3.239,36.784,3.054,36.763,23350,3130,0.180617,-0.058447,-0.002421,-0.024345,0,6,335,12
4,0039Y7A8,2.912,36.707,3.207,36.698,36613,2138,-0.140080,0.041611,0.154813,0.029611,0,1,351,12


In [ ]:
train.describe()

,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
count,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.000000,83924.0,83924.000000,83924.000000,83924.000000
mean,3.052406,36.739358,3.056962,36.737732,13527.821410,1111.697762,-0.002330,-0.000649,0.002330,0.000649,0.0,3.003277,337.940482,11.600603
std,0.096388,0.032074,0.100710,0.032781,9296.716006,563.565486,0.096562,0.031545,0.100980,0.031940,0.0,1.903628,8.700890,0.489777
min,2.807000,36.589000,2.807000,36.596000,1.000000,1.000000,-0.243605,-0.089226,-0.244316,-0.088584,0.0,0.000000,323.000000,11.000000
25%,2.994000,36.721000,2.995000,36.718000,6108.000000,701.000000,-0.060490,-0.019361,-0.057711,-0.017579,0.0,1.000000,331.000000,11.000000
50%,3.046000,36.742000,3.049000,36.742000,11731.500000,1054.000000,-0.009546,-0.003619,-0.006555,-0.003127,0.0,3.000000,338.000000,12.000000
75%,3.095000,36.760000,3.109000,36.760000,19369.000000,1456.000000,0.042026,0.014697,0.053862,0.017761,0.0,5.000000,345.000000,12.000000
max,3.381000,36.820000,3.381000,36.819000,62028.000000,5238.000000,0.326097,0.153482,0.326097,0.146428,0.0,6.000000,353.000000,12.000000


In [ ]:
train.isnull().sum()

ID                 0
Timestamp          0
Origin_lat         0
Origin_lon         0
Destination_lat    0
Destination_lon    0
Trip_distance      0
ETA                0
dtype: int64

In [ ]:
train.shape[0]

83924

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

arr = np.vstack((train[['Origin_lat', 'Origin_lon']].values,
                    train[['Destination_lat', 'Destination_lon']].values))
sample_ind = np.random.permutation(len(arr))
kmeans =MiniBatchKMeans(n_clusters=90, batch_size=10000).fit(arr[sample_ind])

In [ ]:
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['Origin_lat', 'Origin_lon']])

train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['Destination_lat', 'Destination_lon']])

In [ ]:
arr = np.vstack((train[['Origin_lat', 'Origin_lon']].values,
                    train[['Destination_lat', 'Destination_lon']].values))

from sklearn.decomposition import PCA
pca = PCA().fit(arr)
train['pickup_pca0'] = pca.transform(train[['Origin_lat', 'Origin_lon']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['Origin_lat', 'Origin_lon']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['Destination_lat', 'Destination_lon']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['Destination_lat', 'Destination_lon']])[:, 1]

In [ ]:
StartTime = pd.to_datetime(train['Timestamp'], infer_datetime_format=True)

KeyError: ignored

In [ ]:
train['is_peak_traffic'] = [1 if (5<i<9 or 15<i<20) else 0 for i in StartTime.dt.hour]
train['Day_in_week'] = StartTime.dt.dayofweek
train['Day_in_year'] = StartTime.dt.dayofyear
train['Month'] = StartTime.dt.month
train = train.drop('Timestamp', axis=1)
train = train.drop('Timestamp', axis=1)


KeyError: ignored

In [ ]:
train.head()

,ID,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,is_peak_traffic,Day_in_week,Day_in_year,Month
0,000FLWA8,3.258,36.777,3.003,36.718,39627,2784,0.200067,-0.052816,-0.050092,0.024168,0,2,338,12
1,000RGOAM,3.087,36.707,3.081,36.727,3918,576,0.034477,0.029167,0.027070,0.009644,0,1,344,12
2,001QSGIH,3.144,36.739,3.088,36.742,7265,526,0.089057,-0.006805,0.032986,-0.005816,0,5,327,11
3,002ACV6R,3.239,36.784,3.054,36.763,23350,3130,0.180617,-0.058447,-0.002421,-0.024345,0,6,335,12
4,0039Y7A8,2.912,36.707,3.207,36.698,36613,2138,-0.140080,0.041611,0.154813,0.029611,0,1,351,12


In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

X = train.drop(['ETA', 'ID'], axis=1)
y = train['ETA']

print('splitting into test, validation and training sets')
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.01)
X_val, X_test, y_val, y_test = train_test_split(X, y, shuffle=True, test_size=0.5)

model = CatBoostRegressor(
    loss_function='RMSE',
    iterations=20000,
    learning_rate=1.0,
    task_type='CPU'
)

print('training catboost model')
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=200)

splitting into test, validation and training sets
training catboost model
0:	learn: 265.4156856	test: 266.5967890	best: 266.5967890 (0)	total: 70.8ms	remaining: 23m 36s
200:	learn: 135.9288056	test: 136.8686145	best: 136.8686145 (200)	total: 3.75s	remaining: 6m 9s
400:	learn: 119.2045932	test: 120.6330811	best: 120.6330811 (400)	total: 7.5s	remaining: 6m 6s
600:	learn: 108.5423547	test: 110.1844575	best: 110.1844575 (600)	total: 11.2s	remaining: 6m
800:	learn: 100.6323278	test: 102.6130827	best: 102.6130827 (800)	total: 14.9s	remaining: 5m 57s
1000:	learn: 94.3585008	test: 96.4992772	best: 96.4992772 (1000)	total: 18.6s	remaining: 5m 53s
1200:	learn: 88.9691144	test: 91.2224858	best: 91.2224858 (1200)	total: 22.4s	remaining: 5m 49s
1400:	learn: 84.3650706	test: 86.7045340	best: 86.7045340 (1400)	total: 26s	remaining: 5m 45s
1600:	learn: 80.3316721	test: 82.6999454	best: 82.6999454 (1600)	total: 29.7s	remaining: 5m 41s
1800:	learn: 77.0112797	test: 79.3726970	best: 79.3726970 (1800)	tot

In [ ]:
pd.DataFrame({'actual': y_test, 'predicted': model.predict(X_test)})

,actual,predicted
34491,563,558.716951
45698,1041,1061.730461
73682,978,997.550459
47856,1570,1571.474395
29522,1176,1174.108256
...,...,...
47515,1337,1332.051111
23145,2004,1987.891154
41114,589,582.990192
13213,535,539.605752


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, model.predict(X_test)))
print(rms)

28.948033943368134


In [ ]:
TEST_SET = pd.read_csv('/content/drive/My Drive/Zindi/Test.csv')